<a href="https://colab.research.google.com/github/Praveengovianalytics/Memory_Plus_Agency_Your_AI/blob/main/Memory%2BAgency_%3D_AI_That_Understands_you.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Problem Statement
When AI agents operate without memory, they encounter several significant limitations that impact usability, coherence, and effectiveness:

# 🚫 1. Context Amnesia (Short-Term)
Without any form of memory, each user input is treated in isolation—agents can’t recall previous turns, even within the same chat.

Leads to broken conversations: forgotten names, tasks, or preferences mid-session

Example: a trip-planning assistant forgetting earlier details like destination or dates breaks continuity .

#🧠 2. No Personalization or Learning (Long-Term)
Stateless agents lack the ability to learn from past sessions or build user profiles over time.

They can’t remember facts about the user, their preferences, ongoing projects, or past decisions

This results in repetitive prompts (“What’s your birthday again?”) and hinders long-term engagement.

# 🤖 3. Inability to Handle Complex, Multi-Step Tasks
Tasks that require planning or multi-step execution over time are severely impacted.

Without memory, agents can't pause, recall progress, or resume workflows.

Autonomous systems like AutoGPT illustrate this: lacking memory leads to repeated loops or forgotten subtasks

# Memory Basics


# 🧠 1. Semantic Memory
What it is: A long-term knowledge base of facts, definitions, concepts, and user-specific attributes.
When to use it:

To ground responses in factual knowledge (e.g. "Paris is capital of France")

To store persistent user preferences or profiles

To answer queries requiring domain knowledge
Benefits: Adds consistency and enables personalization across sessions
Example: Storing that a user prefers quick recipes or morning updates


# 🪞 2. Episodic Memory
What it is: A collection of past user interactions or specific agent actions.
When to use it:

To maintain context across turns or sessions

To refer back to past decisions ("You asked about Xmas gifts last time")

For case-based reasoning (learn from prior successes/failures)
Benefits: Supports continuity and personalized follow-ups
Example: Reminding the user of their previous project summary

# ⚙️ 3. Procedural Memory
What it is: The agent’s internal workflows, reasoning steps, and instructions—essentially how it operates.
When to use it:

To preserve and reuse refined prompts or reasoning strategies

To enforce task-specific procedures or tool invocation rules

For agents that self-optimize over time
Benefits: Ensures consistency, reliability, and improves over time
Example: Keeping a refined system prompt to handle financial queries

### Demo

### Retrieval-Augmented Generation (RAG) Agent using LangMem for long‑term memory and OpenAI’s Agent SDK for tool-driven scraping, embedding, retrieval, and generation:

# Setup 1

In [1]:
!pip install langmem openai langgraph tiktoken pydantic beautifulsoup4 requests -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.9/66.9 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.4/152.4 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.3/289.3 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 8.3 MB/s eta 0:00:00


In [3]:
from google.colab import userdata
import os
os.environ["OPENAI_API_KEY"] = userdata.get("openai_api_key")

# 🧠 1. Setup LangMem for Long‑Term Memory

In [10]:
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.store.memory import InMemoryStore
from langmem import create_manage_memory_tool, create_search_memory_tool

# 1. Setup your LLM model
model = ChatOpenAI(model="gpt-4-turbo", temperature=0)

# 2. Prepare memory store and tools
store = InMemoryStore(
    index={"dims": 1536, "embed": "openai:text-embedding-3-small"}
)

# Create LangMem tools for semantic, episodic, procedural memory
manage_sem = create_manage_memory_tool(namespace=("semantic",), store=store)
search_sem = create_search_memory_tool(namespace=("semantic",), store=store)

manage_epi = create_manage_memory_tool(namespace=("episodic",), store=store)
search_epi = create_search_memory_tool(namespace=("episodic",), store=store)

manage_proc = create_manage_memory_tool(namespace=("procedural",), store=store)
search_proc = create_search_memory_tool(namespace=("procedural",), store=store)

tools = [
    manage_sem, search_sem,
    manage_epi, search_epi,
    manage_proc, search_proc,
]

# 3. Create a checkpointer for short-term memory across turns
checkpointer = InMemorySaver()

# 4. Build the agent
agent = create_react_agent(
    model=model,
    tools=tools,
    prompt="You are a RAG agent with semantic, episodic, and procedural memory.",
    checkpointer=checkpointer,
    store=store,  # needed so memory tools know where to store
    debug=True
)

In [11]:
config = {"configurable": {"thread_id": "session-123"}}

# Turn 1
out1 = agent.invoke(
    {"messages": [{"role":"user","content":"Hi, remember I prefer dark mode?"}]},
    config
)

# Turn 2: memory retrieved automatically
out2 = agent.invoke(
    {"messages": [{"role":"user","content":"What’s my display preference?"}]},
    config
)

[-1:checkpoint] State at the end of step -1:
{'messages': []}
[0:tasks] Starting 1 task for step 0:
- __start__ -> {'messages': [{'content': 'Hi, remember I prefer dark mode?', 'role': 'user'}]}
[0:writes] Finished step 0 with writes to 1 channel:
- messages -> [{'content': 'Hi, remember I prefer dark mode?', 'role': 'user'}]
[0:checkpoint] State at the end of step 0:
{'messages': [HumanMessage(content='Hi, remember I prefer dark mode?', additional_kwargs={}, response_metadata={}, id='dc79129c-0162-4d88-8a03-e2b5ad170a48')]}
[1:tasks] Starting 1 task for step 1:
- agent -> {'is_last_step': False,
 'messages': [HumanMessage(content='Hi, remember I prefer dark mode?', additional_kwargs={}, response_metadata={}, id='dc79129c-0162-4d88-8a03-e2b5ad170a48')],
 'remaining_steps': 24}
[1:writes] Finished step 1 with writes to 1 channel:
- messages -> [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_lbarBmHN6GVIzxgIP6DK9tYs', 'function': {'arguments': '{"content":"The user 

###  Take Away
With this setup, your agent can remember preferences (semantic), recall past interaction episodes (episodic), and maintain or adapt its behavior guidelines (procedural), all within the ReAct framework. You can also add tools for RAG retrieval such as web scraping or database lookup, embedding and storing in the same InMemoryStore. Want to see how to wire in a scraper for RAG?

# Adding Memory to ScrapAny Web Agent

In [22]:
import requests
from bs4 import BeautifulSoup
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
from langgraph.store.memory import InMemoryStore
from langgraph.checkpoint.memory import InMemorySaver
from langmem import create_manage_memory_tool, create_search_memory_tool

# 🛠️ 1. Define the web scraper tool for RAG
def scrape_url(url: str) -> str:
    """Scrapes visible text from a webpage."""
    resp = requests.get(url, timeout=10)
    soup = BeautifulSoup(resp.text, "html.parser")
    for tag in soup(['script', 'style']):
        tag.decompose()
    return soup.get_text(separator='\n').strip()

# 2. Memory store setup
store = InMemoryStore(index={"dims":1536, "embed":"openai:text-embedding-3-small"})

# 3. Memory tools for semantic, episodic, and procedural memory
sem_m = create_manage_memory_tool(namespace=("semantic",), store=store)
sem_s = create_search_memory_tool(namespace=("semantic",), store=store)
epi_m = create_manage_memory_tool(namespace=("episodic",), store=store)
epi_s = create_search_memory_tool(namespace=("episodic",), store=store)
proc_m = create_manage_memory_tool(namespace=("procedural",), store=store)
proc_s = create_search_memory_tool(namespace=("procedural",), store=store)

# 4. Bundle tools (including scraper)
tools = [scrape_url, sem_m, sem_s, epi_m, epi_s, proc_m, proc_s]  # total 7

# 5. Initialize LLM
llm = ChatOpenAI(model="gpt-4-turbo", temperature=0)

# 6. Create RAG + Memory agent
agent = create_react_agent(
    model=llm,
    tools=tools,
    prompt="You are a RAG agent with memory: can scrape the web and remember facts, episodes, and behavior.",
    checkpointer=InMemorySaver(),
    store=store,
    debug=True
)

In [30]:
import requests
from bs4 import BeautifulSoup
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
from langgraph.store.memory import InMemoryStore
from langgraph.checkpoint.memory import InMemorySaver
from langmem import create_manage_memory_tool, create_search_memory_tool

# 🛠️ 1. Define scraper tool with a docstring
@tool
def scrape_url(url: str) -> str:
    """Scrape the visible text content from a webpage URL."""
    resp = requests.get(url, timeout=10)
    soup = BeautifulSoup(resp.text, "html.parser")
    for tag in soup(["script", "style"]):
        tag.decompose()
    return soup.get_text(separator="\n").strip()

# 2. Memory store and memory tools (semantic, episodic, procedural)
store = InMemoryStore(index={"dims": 1536, "embed": "openai:text-embedding-3-small"})
sem_m = create_manage_memory_tool(namespace=("semantic",), store=store)
sem_s = create_search_memory_tool(namespace=("semantic",), store=store)
epi_m = create_manage_memory_tool(namespace=("episodic",), store=store)
epi_s = create_search_memory_tool(namespace=("episodic",), store=store)
proc_m = create_manage_memory_tool(namespace=("procedural",), store=store)
proc_s = create_search_memory_tool(namespace=("procedural",), store=store)

# 3. Bundle tools
tools = [scrape_url, sem_m, sem_s, epi_m, epi_s, proc_m, proc_s]  # total 7 tools

# 4. Initialize LLM and create RAG + Memory agent
llm = ChatOpenAI(model="gpt-4-turbo", temperature=0)
agent = create_react_agent(
    model=llm,
    tools=tools,
    prompt="You are a RAG agent with memory; you can scrape webpages and remember facts, episodes, and behavior.",
    checkpointer=InMemorySaver(),
    store=store,
    debug=True
)

def chat_cli():
    print("🎯 Welcome! Type 'exit' to quit.")
    config = {"configurable": {"thread_id": "cli-session"}}
    while True:
        user_input = input("You: ").strip()
        if not user_input or user_input.lower() == "exit":
            break
        resp = agent.invoke(
            {"messages": [{"type": "human", "content": user_input}]},
            config
        )
        for msg in resp["messages"]:
            if msg.type == "assistant":
                print("Agent:", msg.content, "\n")
                break

if __name__ == "__main__":
    chat_cli()